In [1]:
import tensorflow as tf
from tensorflow import keras

对于常用的网络模型，如ResNet、VGG等，不需要手动创建网络，可以直接从keras.applications子模块中通过一行代码即可创建并使用这些经典模型，同时还可以通过设置weights参数加载预训练的网络参数，非常方便。

## 1.加载模型

以ResNet50为例，一般去除其最后一层后作为新任务的特征提取器。然后使用ImageNet预训练，并根据自己任务加一层全连接层或其他网络。

代码如下，实现加载ImageNet预训练的ResNet50：

In [2]:
# 加载ImageNet预训练网络模型，并去掉最后一层
resnet=keras.applications.ResNet50(weights='imagenet',include_top=False) # include_top: whether to include the fully-connected layer at the top of the network.
resnet.summary()

resnet1=keras.applications.ResNet50(weights='imagenet',include_top=True)
resnet1.summary()

# 测试网络的输出
x=tf.random.normal([4,224,224,3])
out=resnet(x)
out1=resnet1(x)
print(out.shape)
print(out1.shape)

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

对于某个具体的任务，需要设置自定义的输出节点数，以100类的分类任务为例：<br>
在ResNet50基础上重新构建新网络：新建一个池化层(这里的池化层暂时可以理解为高、宽维度下采样的功能),将特征从\[b, 7,7,2048\]降维到\[b, 2048\]。代码如下:

In [3]:
# 新建池化层
global_average_layer=keras.layers.GlobalAveragePooling2D()
# 新建全连接层
fc=keras.layers.Dense(100)

x=tf.random.normal([4,2048])

# 包裹网络模型
mynet=keras.Sequential([resnet,global_average_layer,fc])
mynet.summary()

(4, 2048)


通过设置`resnet.trainable=False`可以选择冻结ResNet部分的网络参数，只训练新建的网络层。

In [ ]:
import os
pid=os.getpid()
!kill -9 $pid
